In [29]:
%matplotlib inline
import numpy as np
import scipy as sp
import json 
import csv
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [72]:

#dataframe for the finding API, will be merged with the shopping GetSingleItem
df_finding=pd.read_json("./data/completedItems.json")
#df_finding

df_getsingleitem = pd.read_json("./data/auctions_detailed.json")
df_getsingleitem=df.rename(columns = {'ItemID':'itemId'})
#df_getsingleitem.head()

df = pd.merge(df_finding, df_getsingleitem, on='itemId')
df.head()


TypeError: type object argument after * must be a sequence, not itertools.imap

**NEXT, we will delete columns that we just don't need to do our analysis. This could even come at the end once we figure out which aren't useful, but must remember to put it here.**


## Now we will do some Feature engineering

This is to figure out which variables actually make a distinction on profitability

In [67]:
df_finding.shape


(9914, 23)

We delete the columns that are irrelevant to our analysis. For example, DiscountPriceInfo tells us no information about our data, because none of our auctions have discounted prices.

In [32]:
del df_getsingleitem['Charity'], df_getsingleitem['Country'], df_getsingleitem['ConvertedCurrentPrice'], df_getsingleitem['DiscountPriceInfo']
del df_getsingleitem['ListingStatus'], df_getsingleitem['PaymentAllowedSite'], df_getsingleitem['ReserveMet'], df_getsingleitem['SKU'], df_getsingleitem['Site'], df_getsingleitem['Storefront']
del df['Subtitle'], df['TimeLeft'], df['ViewItemURLForNaturalSearch'], df['ConvertedBuyItNowPrice']

We then remove unnecessary information from each of the columns. For example, on CurrentPrice, we only want the value in the dictionary and not the whole dictionary. 

In [33]:
#df['CurrentPrice'].values[3][u'Value']

for i in df.index:
    
    #getting just the float value and ridding of the dictionary 
    df['CurrentPrice'].values[i] = df['CurrentPrice'].values[i][u'Value']
    
    if pd.isnull(df['BuyItNowPrice'].values[i]) == False:
        #print df['BuyItNowPrice'].values[i]
        df['BuyItNowPrice'].values[i] = df['BuyItNowPrice'].values[i][u'Value']
    if pd.isnull(df['MinimumToBid'].values[i]) == False:
        df['MinimumToBid'].values[i] = df['MinimumToBid'].values[i][u'Value']
    
    #we make a numbering system fo ListingType so we can see which type of auction is most successful
    #if df['ListingType'][i] == 'Chinese':
        #df['ListingType'][i] = 0
    #print df.ix[i,'ListingType']
    
    df['ListingType'] = df['ListingType'].map({'Chinese': 0, 'FixedPriceItem': 1})
    
    #if df.ix[i,'ListingType'] == 'Chinese':
    #    df.ix[i,'ListingType'] == 0
    #df.loc[df['ListingType'] == 'Chinese'] = 0
    #df.loc[df['ListingType'] == 'FixedPriceItem'] = 1
    #elif df['ListingType'][i] == 'FixedPriceItem':
        #df['ListingType'][i] == 1
    #if df.ix[i,'ListingType'] == 'FixedPriceItem':
    #    df.ix[i,'ListingType'] == 1
        
    
    
#because each auction we are dealing with is completed, we say
df=df.rename(columns = {'CurrentPrice':'FinalPrice'})

In [57]:
df.shape

(9856, 54)

In [17]:
dftouse = df.copy()

**We can now make some separator variables to make listings unique**

Certain parameters that are zero are can be significant indicators of specific characteristics of an auction. For example, if the HitCount is zero, that means nobody really bothered to interact with the auction at all.

In [18]:
ZERO_IMPORTANT_VARIABLES = ['BidCount', 'HitCount']

To mark binary or indicator variables, we see which of the dataframe fall into this category

In [19]:
for v in df.columns:
    l=df[v].unique()
    if len(l) <= 10:
        print v, l

AutoPay [False True]
AvailableForPickupDropOff [ nan   1.]
BestOfferEnabled [False True]
BuyItNowAvailable [  1.  nan]
EligibleForPickupDropOff [ nan   1.]


TypeError: unhashable type: 'list'